In [40]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
import logging
import json
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
import scipy.sparse as sps
from sklearn.model_selection import KFold
from gensim.models.tfidfmodel import TfidfModel
from itertools import chain
from itertools import count
import nltk
from sklearn import linear_model
from sklearn import model_selection
from nltk.corpus import stopwords
import gc
from itertools import islice
import string

In [42]:
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense, Dropout, Input, Lambda, Merge, merge, LSTM
from keras.optimizers import RMSprop
from keras import backend as K

from keras.preprocessing import sequence
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

In [43]:
logging.basicConfig(level=logging.INFO)

In [44]:
max_len = 20
features = 300

In [45]:
def dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def dot(vects):
    x, y = vects
    return (x-y)**2

input_shape = (max_len+1, features)

# network definition
base_network = Sequential()
base_network.add(LSTM(300, input_shape=input_shape))
base_network.add(Dropout(0.1))
base_network.add(Dense(100, activation='sigmoid'))
#base_network.add(Dropout(0.1))
#base_network.add(Dense(200, activation='relu'))

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(dot, output_shape=dist_output_shape)([processed_a, processed_b])

#distance = merge([processed_a, processed_b], mode="multiply")
print(distance.get_shape())

cosine = Sequential()
cosine.add(Dense(1, init = 'normal', activation = 'sigmoid', input_shape=(100,)))
answer = cosine(distance)

model = Model([input_a, input_b], answer)

# train
rms = RMSprop()

def log_loss(y_true, y_pred):
    return K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))

# loss='binary_crossentropy'
model.compile(loss='mean_squared_error', optimizer='adam')
#model.compile(loss='binary_crossentropy', optimizer=rms)

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

(?, 100)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, input_shape=(100,), kernel_initializer="normal", activation="sigmoid")`


In [69]:
zeros = [np.zeros(features)]
batch_size = 200
def handle(data):
    cuted = data[:max_len]
    padded = zeros* (max_len- len(cuted) + 1 ) + cuted
    stacked = np.vstack(padded)
    return stacked

def Generator():
    while True:
        for part in range(10):
            fname = "cdata/per_word/train_cleaned_nostop_vectorized_perword_{0:d}.pickle".format(part)
            print(fname)
            perword = pd.read_pickle(fname)
            length = perword.shape[0]
            for batch in range((length//batch_size) + 1):
                data = perword[batch*batch_size:(batch+1)*batch_size]
                q1 = np.stack(data.question1.apply(handle).values)
                q2 = np.stack(data.question2.apply(handle).values)
                y = data.is_duplicate.values
                #yield q1.reshape(1,11,300), np.array([y])
                yield [q1, q2], y
                #yield [q1, q1], [y,y]
         

In [70]:
g = Generator()

In [71]:

#model.fit(tx, ty, epochs=3)
model.fit_generator(g, epochs=10, steps_per_epoch = 10000, )

# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

cdata/per_word/train_cleaned_nostop_vectorized_perword_0.pickle
Epoch 1/10
 1032/10000 [==>...........................] - ETA: 5319s - loss: 0.1276

KeyboardInterrupt: 

In [67]:
pom = model.evaluate_generator(Generator(), steps=10)

cdata/per_word/train_cleaned_nostop_vectorized_perword_0.pickle


In [29]:
bp = []
bt = []
for x in g:
    
    bp.append(model2.predict(x[0]))
    bt.append(x[1])
    break

In [30]:
from sklearn import metrics

In [31]:
bt[0]
bp[0][:,0]
metrics.log_loss(bt[0], bp[0][:,0])

0.34108459226195281

In [72]:
def serialize(model, fname):
    f = open(fname, "w")
    f.write(model.to_json())
    model.save_weights(fname+"w")

In [73]:
serialize(model, "models/lstm2")

In [9]:
def deserialize(fname):
    model = model_from_json(open(fname).read())
    model.load_weights(fname+"w")
    return model

In [10]:
model2 = deserialize("models/lstm")

In [74]:
model2 = model

In [75]:
model2.evaluate_generator(g, steps=10)

0.13153744339942933

In [76]:
model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 21, 300)       0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 21, 300)       0                                            
____________________________________________________________________________________________________
sequential_5 (Sequential)        (None, 100)           751300      input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 1)             0           sequential_5[1][0]      

In [79]:
submit_pred = []
submit_ids = []
for part in range(51):
    print(part)
    test = pd.read_pickle("cdata/per_word/test_cleaned_nostop_vectorized_perword_{0:d}.pickle".format(part))
    print("loaded")
    submit_ids.append(test.test_id.values)
    q1 = np.stack(test.question1.apply(handle).values)
    q2 = np.stack(test.question2.apply(handle).values)
    #yield q1.reshape(1,11,300), np.array([y])
    submit_pred.append(model2.predict([q1, q2]))

0
loaded
1
loaded
2
loaded
3
loaded
4
loaded
5
loaded
6
loaded
7
loaded
8
loaded
9
loaded
10
loaded
11
loaded
12
loaded
13
loaded
14
loaded
15
loaded
16
loaded
17
loaded
18
loaded
19
loaded
20
loaded
21
loaded
22
loaded
23
loaded
24
loaded
25
loaded
26
loaded
27
loaded
28
loaded
29
loaded
30
loaded
31
loaded
32
loaded
33
loaded
34
loaded
35
loaded
36
loaded
37
loaded
38
loaded
39
loaded
40
loaded
41
loaded
42
loaded
43
loaded
44
loaded
45
loaded
46
loaded
47
loaded
48
loaded
49
loaded
50
loaded


In [83]:
import gc

In [84]:
gc.collect()

252

In [85]:
predictions = np.array([[0]])
indices = np.array([0])
for i in range(len(submit_ids)):
    predictions = np.concatenate((predictions, submit_pred[i]))
    indices = np.concatenate((indices, submit_ids[i]))

In [86]:
sub = pd.DataFrame({'test_id': indices[1:], 'is_duplicate': predictions[1:].T[0]})
sub.to_csv('lstm2.csv', index=False)
sub.head()

,is_duplicate,test_id
0,0.092538,0
1,0.013683,1
2,0.241158,2
3,0.020488,3
4,0.002837,4


In [82]:
sub.shape

(2345796, 2)